In [22]:
import pickle
import numpy as np
from random import shuffle
import random
from bs4 import BeautifulSoup
import requests
import math
import datetime
import time

In [2]:
f = open('../Results/Pickles/member_names.pickle', 'rb')
member_names = pickle.load(f)
f.close()

In [3]:
f = open('../Results/Pickles/edge_details.pickle', 'rb')
edge_details = pickle.load(f)
f.close()

In [5]:
data = []
n = len(member_names)

for i in range(n):
    for j in range(i + 1, n):
        if (i, j) in edge_details:
            data.append((i, j, 1))
        else:
            data.append((i, j, 0))

In [6]:
indices = [i for i in range(len(data))]
shuffle(indices)

In [14]:
training_data = data[:int(4 * len(data) / 5)]
test_data = data[int(4 * len(data) / 5):]

mat = [[0.0 for i in range(n)] for j in range(n)]
for rec in training_data:
    if rec[2] == 1:
        mat[rec[1]][rec[0]] = 1
        mat[rec[0]][rec[1]] = 1

# Making the second most basic model in order to find good starting values for alpha and beta

In [18]:
alpha = 0.0
beta = [0.0 for i in range(len(member_names))]
lmbda = 1

In [19]:
for i in range(1000):
    alpha = 0.0
    for rec in training_data:
        alpha += rec[2] - beta[rec[0]] - beta[rec[1]]
    alpha /= len(training_data)
    old_beta = beta
    for user1 in range(n):
        beta[user1] = 0.0
        for user2 in range(n):
            if user1 == user2:
                continue
            beta[user1] += mat[user1][user2] - (alpha + old_beta[user2])
        beta[user1] /= (n + lmbda)
    old_beta = beta

In [20]:
np.save('beta_user.npy', beta)
print(alpha)

-2.029422407826445


In [38]:
latent_feature_dim = 7
learning_rate = 0.001
gamma_user = [[random.random() for i in range(n)] for j in range(latent_feature_dim)]

In [39]:
alpha = -2.029422407826445
beta = np.load('beta_user.npy')
gamma_user = np.array(gamma_user)

In [40]:
threshold = 1000000

for iteration in range(1000):
    alpha = 0.0
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            error = mat[i][j] - (alpha + beta[i] + beta[j] + np.dot(gamma_user[:, i].T, gamma_user[:, j]))
            beta[i] += learning_rate * (error - lmbda * beta[i])
            beta[j] += learning_rate * (error - lmbda * beta[j])
            temp1 = learning_rate * (error * gamma_user[:, j] - lmbda * gamma_user[:, i])
            temp2 = learning_rate * (error * gamma_user[:, i] - lmbda * gamma_user[:, j])
            gamma_user[:, i] += temp1
            gamma_user[:, j] += temp2
    delta = 0.0
    for rec in test_data:
        predicted = alpha + beta[rec[0]] + beta[rec[1]] + np.dot(gamma_user[:, rec[0]].T, gamma_user[:, rec[1]])
        print(predicted)
        if predicted > 0.5:
            predicted = 1
        else:
            predicted = 0
        #print('Predicted : ' + str(predicted) + ', Actual : ' + str(rec[2]))
        delta += abs(predicted - rec[2])
    mae = delta / len(test_data)
    print(mae)
    if mae >= threshold:
        break
    threshold = mae

0.0369937948002
0.101239549353
0.0766658621765
0.120738235451
0.0768342569149
0.0935837842808
0.134595689897
0.0923318779121
0.0139623528545
0.0507570607821
0.00532541728178
-0.0144536900299
-0.0260114434759
-0.0181900004322
0.00999490479826
0.0212984655111
-0.0350653901107
0.0190094154489
0.0619482823345
-0.00737667439831
0.117332760556
0.0731134455123
-0.0231649277778
0.000363475601736
-0.0166895073643
-0.0123510357578
-0.0195956154266
-0.019439713355
-0.035249164203
-0.024293733244
-0.0328333492478
-0.00388943304994
-0.000545426597735
-0.0173334898104
-0.00296830012996
0.00717480158387
-0.0195635909568
0.00464713604095
0.0252168486232
0.0576775194157
-0.0339698035731
0.0469303650702
0.0216581946261
-0.0144681660923
0.07043830734
0.000774294295459
-0.00384795177938
0.0109134344455
0.0129544509341
0.0183813166678
0.0308972005242
0.0124688032028
0.0232680321148
0.0378598654354
0.0324343557331
-0.00320937438247
0.0798716155714
0.0287400699172
0.0285252173456
-0.0111131101406
0.001133722

0.0786533510615
0.0478522617368
0.088811128454
-0.0310970620009
0.0780876621426
0.066613688999
-0.0525318050894
0.166429624968
0.0630275029064
-0.0275321471005
-0.0142480406809
0.151593388796
0.0237580588766
0.101100503897
0.135717159783
0.00362380848524
0.148910961785
0.120763467556
-0.0277747712678
0.0185610785637
0.0122143915406
0.112852356689
0.0110705735371
0.166154613942
0.0721055222239
0.05863193616
0.0717449539427
0.089984659771
0.2139273957
0.118641210343
0.0786048051368
0.123831545171
0.124379786889
0.152156266877
0.143102220953
0.127548567209
0.0470114814121
0.147432869913
0.147769744211
0.07263489157
0.0325335795395
0.0718389935836
0.150242395849
0.102216990551
0.0153740433348
0.000614633028127
0.138466498285
0.166308447584
0.106391986381
0.0855921998188
0.0745416221024
0.10290191833
0.11023180805
0.0979782799923
0.228393418849
0.12657723084
0.0429034141203
0.140168937377
0.103976065558
0.17169150134
0.113782163623
0.127675887337
0.0743701861551
0.108754598965
0.09690129949

0.0520576935573
0.0721319832639
0.0812552156031
0.0321943999872
0.10658739298
0.0712054712904
0.00966082079917
0.00933583621237
-0.00872455517313
-0.0289007394407
-0.00370424573666
0.0140304781687
0.0331077521479
-0.0224837578261
0.0454741390993
-0.000910816671904
-0.0382980970512
0.034559758144
-0.0186381694919
0.024142337242
0.0296689665292
-0.0426530203152
0.00106002785869
0.0306459145445
0.060346986295
0.033296022912
0.0403446909077
0.037591822227
0.0367408603708
0.0330412435515
0.00424175576685
-0.0104360593949
0.0614577621663
-0.00267489809803
-0.0048256434211
0.0154152414373
-0.00698079499346
0.00298180256438
-0.0435420847272
0.0542981472248
0.107334885117
-0.0144951542184
-0.00797099262852
0.0544804360535
0.0291550126893
0.0218798860728
0.0275011391167
0.0228293500898
0.00670956662576
0.066007195458
0.0789824179526
0.0155915860659
0.0634156434644
0.102985721292
0.0437918544045
0.0515650379647
0.120370398005
0.0895580824319
-0.0152010616833
0.0566814689263
0.0709862557774
0.0232

0.0794194446502
0.0153306167043
0.0271936290122
0.0233767164472
0.0361204827386
-0.0267955864244
-0.0325822481741
0.0195477531467
0.030729483371
0.033467336761
0.0404642934657
0.0725185917664
0.000511597447938
-0.0336787861284
-0.0240558349949
0.0488940549741
-0.0285474785627
-0.0224173454304
-0.0430730581371
-0.0430249001479
-0.0153099674539
0.0572974359229
0.0232017859927
-0.0116683088577
-0.0176242876436
-0.0408808193027
-0.0284713773023
-0.00253796919539
-0.0397131994801
-0.0225587743304
-0.00609227598834
0.076559264626
0.0752402085197
0.0909624156907
0.0576854760407
0.0681180316991
0.113653059964
0.0720501486577
-0.00384710075963
0.0622744078708
0.0434512449431
-0.0184824870087
-0.0316146119366
-0.00713865953499
0.026658830712
-0.0147106562101
0.00178360066542
0.023182545914
0.0733383828887
-0.0167733905474
0.10068381583
0.059526787892
-0.0183014001384
-0.00519506169721
-0.0380657480838
-0.0344598072404
-0.0260282025908
-0.00794301421517
-0.0367573437756
-0.0149715228667
0.0104301

0.127609564879
0.112629309336
0.0130535920733
0.100427936378
0.150226033539
0.125892765569
0.00110375710292
0.00598007698578
0.146419673061
0.141217944374
0.121256626234
0.0815130919062
0.0514239375957
0.114524898045
0.115800559411
0.130557707719
0.173962458113
0.103824623687
0.0648227929814
0.136891739655
0.119486016019
0.164977200642
0.122518947897
0.129084840461
0.0688142597841
0.136624894688
0.130020338562
0.143600321465
0.0856492544769
0.0830716626426
0.108623567871
0.117055662117
0.101566272001
0.152761016652
0.0569328010828
0.0531434570427
0.0934796963774
0.107362240799
0.0795762388844
0.11984770707
0.0829925970605
0.164920951266
0.0638298344121
0.0341199534677
0.0185560011205
0.0812073620501
0.0363602539113
0.106226651101
0.0754921503651
0.065926640255
0.0128649810981
0.0753535448516
0.141704357038
-0.000258015314989
0.073757401561
0.0488835554141
0.0585272460013
0.0532982309378
0.0540928804649
0.105098800016
0.0924337361631
0.119725829928
0.132214055499
0.148957124948
0.180383

...

-0.0189434086103
-0.0201615880074
-0.0259619289639
-0.0289719656077
-0.0201595941483
-0.00672709370353
-0.00426387359175
-0.0137168107813
0.0575796497619
-0.00808003937911
-0.00449650485351
0.056325858839
0.0501041206676
-0.0127882864276
-0.0270568744048
-0.00829761056651
-0.0189928742834
-0.0155275880107
0.153772491876
0.150273540882
0.145075249433
0.146378307083
0.143057135468
0.145824510113
0.1251623799
0.148328786054
0.0948109372619
0.165672565454
0.14479517111
0.0940397540486
0.0395543418683
0.143857476279
0.172907032305
0.128384670956
0.0305938836488
0.0231113530028
0.131658290544
0.142649822804
0.157078098024
0.0851785925752
0.0919810494487
0.13226191266
0.129478931375
0.1082441151
0.184876140353
0.146823447472
0.0773393843873
0.119328845048
0.116501786399
0.1126999149
0.149987789841
0.146197815898
0.0767623526507
0.124190009763
0.125855414792
0.16907392609
0.102721260125
0.0998629864885
0.13433861542
0.10054958477
0.105867521506
0.119115538341
0.0988778431168
0.0586035113789
0.

0.133882163917
0.0624488548574
0.111307043626
0.112059452554
0.1543878707
0.0920545091467
0.0895283129002
0.116500823669
0.0921627376198
0.0920525503436
0.108593406696
0.0812460542403
0.0471115207767
0.0605397744559
0.0867894396468
0.0819044215067
0.0956707694722
0.0882782235746
0.135961515944
0.0483281004379
0.0328283114951
0.0341261827863
0.0860096704266
0.0327077149656
0.0514129895779
0.0319667094969
0.0323284144727
0.0298915525261
0.0889523337513
0.0879479744048
0.0208762450954
0.0469068386544
0.0265090572626
0.0367762891864
0.0411637924786
0.0266833005551
0.0454311089212
0.0525183078116
0.103318700419
0.11368437039
0.122585771144
0.107319903956
0.107959969355
0.140899999848
0.119991290724
0.0413974208775
0.107300494712
0.0783394274098
0.0362103353452
0.039421176163
0.036746622427
0.0588964319284
0.032231712938
0.0469558518201
0.0747597382542
0.0941434009369
0.0512457124354
0.131540402999
0.0920792516479
0.0240089714688
0.0512485680446
0.0196282391728
0.0226916236659
0.024204681941

-0.0262761502307
-0.0244095199774
-0.02542450746
-0.0268828189992
-0.0320181039474
-0.0224321843148
-0.0137559082808
-0.00644709469269
-0.0181313016837
0.0527149041814
-0.0145220070489
-0.0131334486915
0.0530067708784
0.0428536506441
-0.0190382701275
-0.0277225671002
-0.0124205408951
-0.0270251359462
-0.02066883409
0.108168265404
0.151658535692
0.0810766908953
0.0789682344008
0.116812893862
0.0865377222379
0.0846462797929
0.10597499955
0.0749040985533
0.0418997880291
0.0532724921079
0.0804459425932
0.0711836801479
0.0910712979394
0.0784454876245
0.137156133411
0.0508716088003
0.0215067183805
0.0318171092603
0.077601353711
0.0311376017864
0.0505804249082
0.0292091823361
0.0305801563444
0.028170575643
0.0833317863173
0.0893558707661
0.0141235610234
0.045971375255
0.0166004361781
0.0268876921549
0.0384714466265
0.0267397669576
0.0394397881926
0.0540717000069
0.108869703256
0.113210301162
0.122096767513
0.10086735437
0.102272678919
0.144702749302
0.116916073133
0.0340150394619
0.1001204987

0.0246798818
0.0171675007104
-0.0410206477339
-0.0543643436092
-0.0350023600539
-0.0540743194206
-0.0475444641456
-0.0195268625866
-0.0287668341268
-0.0388279206289
0.0198247312456
0.0312985744569
-0.0440715957488
-0.0151577630419
-0.0383261654885
-0.0333127624009
-0.0193778491411
-0.0317586301584
-0.0149630756598
-0.00370331761679
0.046129518677
0.0489999276638
0.0635882117582
0.0445609209096
0.0519869046331
0.0835516742379
0.0606207444653
-0.0203287243122
0.0361882669651
0.00631062021412
-0.0316839286232
-0.0283755907433
-0.029582688541
-0.00552942909914
-0.0220313247882
-0.0262915036251
0.00657450449901
0.0299163937235
-0.0204584432592
0.0697916064736
0.0316788768016
-0.0401603501148
-0.0196212301135
-0.0448282273941
-0.0434564447027
-0.0377314217962
-0.0410374630262
-0.0372332821539
-0.0424191636245
-0.03910319103
-0.0338889736186
-0.0389416353301
-0.038409482068
-0.0391262979702
-0.0286236147602
-0.0378676236852
-0.0416658101213
-0.0227664320563
-0.000985031970907
-0.042005561505


-0.0498612796411
-0.0225255271146
-0.0117612996064
0.0129844425701
0.0412752742768
-0.0126463856266
0.0745713483525
0.0364465455959
-0.0355796473461
-0.00971152934494
-0.0377236531041
-0.0413053185098
-0.0305042593746
-0.0316248085783
-0.0270235000914
-0.0321747327983
-0.0251404808364
-0.0331830290647
-0.0338685166876
-0.0288784541858
-0.0305170636703
-0.0222206287612
-0.0268111171028
-0.0373447427365
-0.0245972782361
-0.00284799148493
-0.0316176592273
0.00331495292267
-0.0120302983745
-0.0302058705842
0.00185146984679
-0.0268951099276
-0.0324922377663
-0.0342033238606
-0.0205692709512
-0.0207532674488
-0.030351004073
-0.0305303835611
-0.0266942252914
-0.0287557890301
-0.0298298633344
-0.029697485138
0.0324432260397
-0.0337708315105
-0.0347067203038
-0.0305588752752
-0.0302834758781
-0.0283849779167
-0.029799802407
-0.0357412384966
-0.0341092051072
-0.0272218548393
-0.0175864130658
-0.0100072303242
-0.0177085369431
0.0448693396108
-0.0230407253532
-0.0165164082419
0.0487919368868
0.038